# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
from holoviews import opts
import warnings
warnings.filterwarnings("ignore")


# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data1/cities1.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hermanus,-34.4187,19.2345,142.116667,67,26,4.24,ZA,1681447677
1,1,berlevag,70.8578,29.0864,135.083333,59,37,7.20,NO,1681447678
2,2,rikitea,-23.1203,-134.9692,147.533333,81,100,10.37,PF,1681447678
3,3,casas grandes,30.3667,-107.9500,141.666667,28,0,7.73,MX,1681447518
4,4,mmathubudukwane,-24.6000,26.4333,143.833333,48,0,0.94,BW,1681447679


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.7,
    color="City",
    hover_cols=["Lat", "Lng", "City", "Humidity"]
)

#map_plot1.opts(width=800, height=600)

# Display the map


map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
ideal_conditions = (city_data_df["Max Temp"] >90 ) & (city_data_df["Wind Speed"] < 2.5) & (city_data_df["Cloudiness"] == 0)
ideal_cities_df = city_data_df[ideal_conditions]

#filtered_cities_df = city_data_df[(city_data_df["Max Temp"] > max_temp) & (city_data_df["Cloudiness"] == cloudiness)]


# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df.dropna()


# Display sample data
# YOUR CODE HERE
ideal_cities_df.head()





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,mmathubudukwane,-24.6000,26.4333,143.833333,48,0,0.94,BW,1681447679
65,65,jamestown,42.0970,-79.2353,143.305556,62,0,0.45,US,1681447707
77,77,coquimbo,-29.9533,-71.3436,141.666667,94,0,1.54,CL,1681447716
80,80,viedma,-40.8135,-62.9967,138.288889,65,0,2.40,AR,1681447717
87,87,ojinaga,29.5667,-104.4167,148.711111,19,0,0.00,MX,1681447720


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,mmathubudukwane,BW,-24.6000,26.4333,48,
65,jamestown,US,42.0970,-79.2353,62,
77,coquimbo,CL,-29.9533,-71.3436,94,
80,viedma,AR,-40.8135,-62.9967,65,
87,ojinaga,MX,29.5667,-104.4167,19,
138,hailey,US,43.5196,-114.3153,74,
148,belton,US,38.8120,-94.5319,62,
158,hovd,MN,48.0056,91.6419,11,
161,bethel,US,41.3712,-73.4140,45,
177,tallahassee,US,30.4383,-84.2807,95,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel

radius = 10000
limit=20
#filters = f"circle:{longitude},{latitude},{radius}"
#bias = f"proximity:{longitude},{latitude}"
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat =hotel_df.loc[index, "Lat"]
    lng = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{lng},{lat},{radius}"

    params["bias"] = f"proximity:{lng},{lat}"
        #params["categories"] = f"accomodation.hotel.{filter}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mmathubudukwane - nearest hotel: Makanyane Lodge
jamestown - nearest hotel: DoubleTree Jamestown
coquimbo - nearest hotel: Hotel Iberia
viedma - nearest hotel: Níjar
ojinaga - nearest hotel: Hotel
hailey - nearest hotel: Fairfield Inn
belton - nearest hotel: Hampton Inn & Suites
hovd - nearest hotel: Khovd hotel
bethel - nearest hotel: Hampton Inn Danbury
tallahassee - nearest hotel: The Governors Inn
saint-pierre - nearest hotel: Tropic Hotel
gat - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel found
tuatapere - nearest hotel: No hotel found
pecos - nearest hotel: Holiday Inn Express & Suites Pecos
bundaberg - nearest hotel: Young Australian Hotel
hamilton - nearest hotel: North Vista Manor
kasane - nearest hotel: Kasane Travel Lodge
port said - nearest hotel: The New Continental Hotel
palmer - nearest hotel: No hotel found
mayo - nearest hotel: The Westin
vila velha - nearest hotel: Hotel Prainha
vallenar - nearest hotel: Hotel del Marqués
lewist

,City,Country,Lat,Lng,Humidity,Hotel Name
4,mmathubudukwane,BW,-24.6000,26.4333,48,Makanyane Lodge
65,jamestown,US,42.0970,-79.2353,62,DoubleTree Jamestown
77,coquimbo,CL,-29.9533,-71.3436,94,Hotel Iberia
80,viedma,AR,-40.8135,-62.9967,65,Níjar
87,ojinaga,MX,29.5667,-104.4167,19,Hotel
138,hailey,US,43.5196,-114.3153,74,Fairfield Inn
148,belton,US,38.8120,-94.5319,62,Hampton Inn & Suites
158,hovd,MN,48.0056,91.6419,11,Khovd hotel
161,bethel,US,41.3712,-73.4140,45,Hampton Inn Danbury
177,tallahassee,US,30.4383,-84.2807,95,The Governors Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
hotel_map=hotel_df.hvplot.points("Lng","Lat",geo=True, tiles ="OSM", frame_width=800, frame_height=600,
                                 size="Humidity", scale=2, 
                                 color="City", 
                                 hover_cols = ["Lat","Lng","City","Humidity","Hotel Name","Country"]
            )

# Display the map
hotel_map 

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)